In [1]:
# prepare the workspace
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# read the data
artworks = pd.read_table("beautified_boxes.tsv")

In [ ]:
"""
prepare the data
"""

In [ ]:
# split the data into training and test sets
# remember that 650/1000 are labeled yet. Either label remaining ones or shorten the set
# training data  is for training the algorithm 
# test set is for evaluating the algorithm
# both must be strictly separated

# features are type, district, environment, countArtists, experience, replaced, content, user rating
# label is "approved"

In [ ]:
# visualisations
# to see range of values, outliers and so on (or actually less in my example, because the values are categorical lol)
# try to see if a potential algorithm is likely to be able to classify the data -> a well defined cluster should be visible

# good plots to visualise the data: 
# 1) feature-pair plot
# their code:
from matplotlib import cm
cmap = cm.get_cmap("gnuplot")
scatter = pd.scatter_matrix(X_train, c = y_train, marker = "o", s = 40, hist_kwds = {"bins":15}, figsize = (12, 12), cmap = cmap)

In [ ]:
"""
ML algorithm options
-k-nearest neighbors: instance or memory based learning
Would likely work for my data set
Likely pretty easy: k=1 (one nearest neighbor)
It needs: a distance metric, k (how many nearest neighbors), weighting function on the neighbor points (not all neigh. have same influence), method for aggregating classes of neighbor points (how to combine influence and decide)
Distance: scikit uses euclidian distance per default
Neighbors: e.g. 5 (odd -> no tie -> no weighting necessary)
Weighting: not necessary, when k/2=0.5
Aggregation: Majority vote
Use-case/code in the video: about knn
"""